In [1]:
from PIL import Image
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.utils import make_grid, save_image
from glob import glob
import cv2
import random

import shutil
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,plot_confusion_matrix

import time
import copy

In [2]:
def seed(seed = 42):
    random.seed(seed) # python random seed 고정
    os.environ['PYTHONHASHSEED'] = str(seed) # os 자체의 seed 고정
    np.random.seed(seed) # numpy seed 고정 
    torch.manual_seed(seed) # torch seed 고정
    torch.cuda.manual_seed(seed) # cudnn seed 고정
    torch.backends.cudnn.deterministic = True # cudnn seed 고정(nn.Conv2d)
    torch.backends.cudnn.benchmark = False # CUDA 내부 연산에서 가장 빠른 알고리즘을 찾아 수행

## DataLoader worker에 대한 seed 설정
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)
    
seed()

## Model Load(ResNet50(pretrained O))

In [3]:
from torchvision import models
import torch
import timm
# m = timm.create_model('seresnet50', pretrained=True)

seresnet50_pretrained = timm.create_model('seresnet50', pretrained=True)
print(seresnet50_pretrained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [4]:
num_classes = 1
num_features = seresnet50_pretrained.fc.in_features
seresnet50_pretrained.fc = nn.Linear(num_features, num_classes)

seresnet50_pretrained

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [6]:
GPU_NUM = 0
os.environ["CUDA_VISIBLE_DEVICES"]='1'
print('gpu? ', torch.cuda.is_available())
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Current gpu: ', torch.cuda.current_device())

if device.type == 'cuda':
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')
    
model = seresnet50_pretrained.to(device)
x = torch.randn(3, 3, 224, 224).to(device)
output = model(x)
print(output)

gpu?  True
Current gpu:  0
Allocated: 0.0 GB
Cached:    0.0 GB
tensor([[-0.1496],
        [ 0.0145],
        [-0.2150]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [7]:
model_name = 'seresnet50(b=16,Adam,Focal_alpha(0.75),WRS,sche,seed)_weights_pt'
model_path = '/mnt/nas100_vol2/LeeJungHoon/AOV_task(binary_clssification)/Model_V2/weights_file/'

In [8]:
model = seresnet50_pretrained.to(device)
model.load_state_dict(torch.load(model_path + model_name))

<All keys matched successfully>

In [9]:
pwd

'/mnt/nas100_vol2/LeeJungHoon/AOV_task(binary_clssification)/Model_V2/Grad_CAM_code'

In [19]:
# import datetime
# current_time = datetime.datetime.now() + datetime.timedelta(hours= 9)
# current_time = current_time.strftime('%Y-%m-%d-%H:%M')


saved_loc = os.path.join('/mnt/nas100_vol2/LeeJungHoon/AOV_task(binary_clssification)/Model_V2/Grad-CAM_results/Grad-CAM_results(seResnet_Focal(0.75)_WRS_sch_validation)', )
if os.path.exists(saved_loc):
    shutil.rmtree(saved_loc)
os.mkdir(saved_loc)

print("결과 저장 위치: ", saved_loc)

결과 저장 위치:  /mnt/nas100_vol2/LeeJungHoon/AOV_task(binary_clssification)/Model_V2/Grad-CAM_results/Grad-CAM_results(seResnet_Focal(0.75)_WRS_sch_validation)


## test dataset & test dataloader

In [12]:
train_normal_path = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v3/data_v3/normal_train_v3/*.jpg'
train_abnormal_path ='/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v3/data_v3/abnormal_train_v3/*.jpg'
valid_normal_path ='/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v3/data_v3/normal_val_v3/*.jpg'
valid_abnormal_path ='/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v3/data_v3/abnormal_val_v3/*.jpg'
test_normal_path ='/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v3/data_v3/normal_test_v3/*.jpg'
test_abnormal_path ='/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v3/data_v3/abnormal_test_v3/*.jpg'

In [20]:
train_normal_glob = glob(train_normal_path)
train_abnormal_glob = glob(train_abnormal_path)
val_normal_glob = glob(valid_normal_path)
val_abnormal_glob = glob(valid_abnormal_path)
test_normal_glob = glob(test_normal_path)
test_abnormal_glob = glob(test_abnormal_path)

print('train_normal :', len(train_normal_glob))
print('val_normal :', len(val_normal_glob))
print('test_normal :', len(test_normal_glob))
print('------------------------------------')
print('train_abnormal :', len(train_abnormal_glob))
print('val_abnormal :', len(val_abnormal_glob))
print('test_abnormal :', len(test_abnormal_glob))

train_normal : 472
val_normal : 52
test_normal : 66
------------------------------------
train_abnormal : 259
val_abnormal : 33
test_abnormal : 39


In [21]:
class Aov_Dysplasia_dataset(Dataset):
    def __init__(self, normal_path, abnormal_path, transform=None):
        #생성자, 데이터를 전처리 
        self.normal_path_list = glob(normal_path)
        self.abnormal_path_list = glob(abnormal_path)
        print(len(self.normal_path_list))
#         self.mode = mode 
    
#         label = np.array([[0, 1], [1, 0]], dtype=np.float32)
        
#         self.label_list = []
#         for i in self.normal_path_list:
#             self.label_list.append(label[0])
            
#         for i in self.abnormal_path_list:
#             self.label_list.append(label[1])
            
        label_policy = {
            'normal': 0, 
            'abnormal': 1
        }
    
        self.label_list= []
        
        for i in self.normal_path_list:
            self.label_list.append(label_policy["normal"])
            
        for i in self.abnormal_path_list:
            self.label_list.append(label_policy["abnormal"])
        
        self.total_img_path_list = self.normal_path_list + self.abnormal_path_list
        self.transform = transform
    
    def __len__(self):
        return len(self.total_img_path_list)
    
    def __getitem__(self, idx):
        
        img = cv2.imread(self.total_img_path_list[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # img = np.array(img, dtype=np.float32)
        #들어오는 이미지의 컬러 형태가 BGR인지 RGB인지 모르기때문에 변형

        
        label = self.label_list[idx]
        
        if self.transform is not None:
            transformed = self.transform(image=img)
            img = transformed['image'] 
            img = torch.tensor(np.array(img), dtype=torch.float32)
            # img = torch.FloatTensor(img)
            img = (img - torch.min(img)) / (torch.max(img)-torch.min(img))
            
            return {'img': img, 'label': label, 'filename': self.total_img_path_list[idx]}
        
        else:
            # img = transformed['image']
            img = torch.tensor(np.array(img), dtype=torch.float32)
            # img = torch.FloatTensor(img)
            img = (img - torch.min(img)) / (torch.max(img)-torch.min(img))
            return{'img': img, 'label': label}

In [22]:
#https://albumentations.ai/docs/api_reference/augmentations/transforms/
import albumentations as A 
from  albumentations.pytorch import ToTensorV2

train_transform = A.Compose(
    [
        # Contrast Limited Adaptive Histogram Equalization 적용
#     A.CLAHE(p=1,clip_limit=(1, 3)),
#     A.HorizontalFlip(p=0.3),
    A.Resize(224,224, interpolation = cv2.INTER_AREA),
    A.ShiftScaleRotate(shift_limit=0, scale_limit=(0.1, 0.2), rotate_limit=0, p=0.6, border_mode=cv2.BORDER_REPLICATE),
    A.CLAHE(clip_limit=(1, 2), p=0.6),
    A.RandomRotate90(p=0.7),
    A.VerticalFlip(p=0.7),
    A.HorizontalFlip(p=0.7),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.3),
    # A.Normalize()
    ToTensorV2()
    ])

valid_transform = A.Compose(
    [
        A.Resize(224,224, interpolation = cv2.INTER_AREA),
        ToTensorV2()
    ])

test_transform = A.Compose(
    [
        A.Resize(224,224, interpolation = cv2.INTER_AREA),
        ToTensorV2()
    ])

In [23]:
train_dataset = Aov_Dysplasia_dataset(train_normal_path, train_abnormal_path, transform = train_transform)
valid_dataset = Aov_Dysplasia_dataset(valid_normal_path, valid_abnormal_path, transform = valid_transform)
test_dataset = Aov_Dysplasia_dataset(test_normal_path, test_abnormal_path, transform = test_transform)

472
52
66


In [24]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle = False ,worker_init_fn=seed_worker)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle = True, worker_init_fn=seed_worker)

In [25]:
# final conv layer name 
# feature map을 추출할 layer를 설정
finalconv_name = 'layer4'

# inference mode
model.eval()

# number of result
# num_result = 10


feature_blobs = []

backward_feature = []

# output으로 나오는 feature를 feature_blobs에 append하도록
def hook_feature(module, input, output):
    feature_blobs.append(output.cpu().data.numpy())
    
# Grad-CAM
def backward_hook(module, input, output):
    backward_feature.append(output[0])

    
model._modules.get(finalconv_name).register_forward_hook(hook_feature)
model._modules.get(finalconv_name).register_backward_hook(backward_hook)


# get the sigmoid weight
params = list(model.parameters())
weight_sigmoid = np.squeeze(params[-2].cpu().detach().numpy()) # [1, 512]


# generate the class activation maps
def returnCAM(feature_conv, weight_simoid, class_idx):
    size_upsample = (224, 224)
    _, nc, h, w = feature_conv.shape # nc : number of channel, h: height, w: width
    output_cam = []
    # weight 중에서 class index에 해당하는 것만 뽑은 다음, 이를 conv feature와 곱연산
    cam = weight_sigmoid[class_idx].dot(feature_conv.reshape((nc, h*w))) 
    cam = cam.reshape(h, w)
    cam = cam - np.min(cam)
    cam_img = cam / np.max(cam)
    cam_img = np.uint8(255 * cam_img)
    output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam


incorrect_abnormal_list =[]
incorrect_normal_list = []

for data in valid_dataloader:
    
    inputs = data['img'].float()
    labels = data['label']
    idx_list = data['filename']
    idx = ' '.join(s for s in idx_list)
    idx = idx.split('/')[-1]
    
    # 모델의 input으로 주기 위한 image는 따로 설정
    image_for_model = inputs.clone().detach()

    # Image denormalization, using mean and std that i was used.
#     image[0][0] *= 0.2257
#     image[0][1] *= 0.2209
#     image[0][2] *= 0.2212
    
#     image[0][0] += 0.4876
#     image[0][1] += 0.4544
#     image[0][2] += 0.4165
    

    # 모델의 input으로 사용하도록.
    image_tensor = image_for_model.to(device)
    logit = model(image_tensor)
    output = torch.squeeze(logit)
    output_sig = torch.sigmoid(output)
    y_pred = output_sig.cpu()
    print(y_pred)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    
    if y_pred != labels.cpu():
        
        if labels.cpu() == 1:
            incorrect_abnormal_list.append(idx)
        else:
            incorrect_normal_list.append(idx)
    label_out = labels.item()
    y_pred_out = y_pred.item()
    print("True label : %d, Predicted label : %d, idx : %s" % (label_out, y_pred_out, idx))
    
    # ============================= #
    # ==== Grad-CAM main lines ==== #
    # ============================= #
    
    score = logit.squeeze() # 예측값 y^c
    score.backward(retain_graph = True) # 예측값 y^c에 대해서 backprop 진행
    
    activations = torch.Tensor(feature_blobs[0]).to(device) # (1, 512, 7, 7), forward activations
    gradients = backward_feature[0] # (1, 512, 7, 7), backward gradients
    b, k, u, v = gradients.size()
    
    # view() 함수에서 -1은 다른 dimension에서 자동적으로 추론되는 것을 의미한다. 
    alpha = gradients.view(b, k, -1).mean(2) # (1, 512, 7*7) => (1, 512), feature map k의 'importance'
    weights = alpha.view(b, k, 1, 1) # (1, 512, 1, 1)
    
    #위에서 지정해준 layer에서의 output인 activations과 backward gradients를 평균한 값인 weights를 곱해준다.
    grad_cam_map = (weights*activations).sum(1, keepdim = True) # alpha * A^k = (1, 512, 7, 7) => (1, 1, 7, 7)
    
    # Apply R e L U
    grad_cam_map = F.relu(grad_cam_map) 
    
    grad_cam_map = F.interpolate(grad_cam_map, size=(224, 224), mode='bilinear', align_corners=False) # (1, 1, 224, 224)
    map_min, map_max = grad_cam_map.min(), grad_cam_map.max()
    grad_cam_map = (grad_cam_map - map_min).div(map_max - map_min).data # (1, 1, 224, 224), min-max scaling

    # grad_cam_map.squeeze() : (224, 224)
    grad_heatmap = cv2.applyColorMap(np.uint8(255 * grad_cam_map.squeeze().cpu()), cv2.COLORMAP_JET) # (224, 224, 3), numpy 
    grad_heatmap = torch.from_numpy(grad_heatmap).permute(2, 0, 1).float().div(255) # (3, 224, 224)
    b, g, r = grad_heatmap.split(1)
    grad_heatmap = torch.cat([r, g, b]) # (3, 244, 244), opencv's default format is BGR, so we need to change it as RGB format.

    # save_image(grad_heatmap, os.path.join(saved_loc, "%d_%d_%s" % (label_out, y_pred_out, idx_out)))
    
    # print(grad_heatmap.type)
    # print(inputsinputs.cpu().type)
    grad_result = grad_heatmap + inputs.cpu() # (1, 3, 224, 224)
    # print(grad_result.shape)
    grad_result = grad_result.div(grad_result.max()).squeeze() # (3, 224, 224)
    
    # save_image(grad_result, os.path.join(saved_loc, "GradCAM&image_%d.jpg" % (xhch_idx+1)))
    
    
    image_list = []
    
    image_list.append(torch.stack([inputs.squeeze().cpu(), grad_heatmap, grad_result], 0)) # (3, 3, 224, 224)
    
    images = make_grid(torch.cat(image_list, 0), nrow = 3)
    
    # image 저장
    save_image(images, os.path.join(saved_loc, "%d_%d_%s" % (label_out, y_pred_out, idx)))
    
    # if  batch_idx + 1 == num_result:
    #     break
        
    feature_blobs.clear()
    backward_feature.clear()

feature_blobs.clear()
backward_feature.clear()


tensor(0.2033, grad_fn=<ToCopyBackward0>)
True label : 0, Predicted label : 0, idx : 13229441__2021-05-07__05d.jpg
tensor(0.7467, grad_fn=<ToCopyBackward0>)
True label : 1, Predicted label : 1, idx : 35122850__2021-09-10__05d.jpg
tensor(0.9121, grad_fn=<ToCopyBackward0>)
True label : 1, Predicted label : 1, idx : 18363043__2018-09-15__05d.jpg
tensor(0.5503, grad_fn=<ToCopyBackward0>)
True label : 1, Predicted label : 1, idx : 33818838__2018-10-16__05d.jpg
tensor(0.1212, grad_fn=<ToCopyBackward0>)
True label : 0, Predicted label : 0, idx : 14485136__2022-04-28__05d.jpg
tensor(0.7617, grad_fn=<ToCopyBackward0>)
True label : 1, Predicted label : 1, idx : 18363043__2018-09-15__305d.jpg
tensor(0.8772, grad_fn=<ToCopyBackward0>)
True label : 1, Predicted label : 1, idx : 37022026__2018-08-29__205d.jpg
tensor(0.5960, grad_fn=<ToCopyBackward0>)
True label : 0, Predicted label : 1, idx : 19590019__2022-05-06__205d.jpg
tensor(0.0402, grad_fn=<ToCopyBackward0>)
True label : 0, Predicted label : 0

In [27]:
# final conv layer name 
# feature map을 추출할 layer를 설정
finalconv_name = 'layer4'

# inference mode
model.eval()

# number of result
# num_result = 10


feature_blobs = []

backward_feature = []

# output으로 나오는 feature를 feature_blobs에 append하도록
def hook_feature(module, input, output):
    feature_blobs.append(output.cpu().data.numpy())
    
    
# Grad-CAM
def backward_hook(module, input, output):
    backward_feature.append(output[0])

    
model._modules.get(finalconv_name).register_forward_hook(hook_feature)
model._modules.get(finalconv_name).register_backward_hook(backward_hook)


# get the sigmoid weight
params = list(model.parameters())
weight_sigmoid = np.squeeze(params[-2].cpu().detach().numpy()) # [1, 512]


# generate the class activation maps
def returnCAM(feature_conv, weight_simoid, class_idx):
    size_upsample = (224, 224)
    _, nc, h, w = feature_conv.shape # nc : number of channel, h: height, w: width
    output_cam = []
    # weight 중에서 class index에 해당하는 것만 뽑은 다음, 이를 conv feature와 곱연산
    cam = weight_sigmoid[class_idx].dot(feature_conv.reshape((nc, h*w))) 
    cam = cam.reshape(h, w)
    cam = cam - np.min(cam)
    cam_img = cam / np.max(cam)
    cam_img = np.uint8(255 * cam_img)
    output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam


incorrect_abnormal_list =[]
incorrect_normal_list = []

for data in test_dataloader:
    
    inputs = data['img']
    labels = data['label']
    idx_list = data['filename']
    idx = ' '.join(s for s in idx_list)
    idx = idx.split('/')[-1]
    
    # 모델의 input으로 주기 위한 image는 따로 설정
    image_for_model = inputs.clone().detach()

    # Image denormalization, using mean and std that i was used.
#     image[0][0] *= 0.2257
#     image[0][1] *= 0.2209
#     image[0][2] *= 0.2212
    
#     image[0][0] += 0.4876
#     image[0][1] += 0.4544
#     image[0][2] += 0.4165
    

    # 모델의 input으로 사용하도록.
    image_tensor = image_for_model.to(device)
    logit = model(image_tensor)
    output = torch.squeeze(logit)
    output_sig = torch.sigmoid(output)
    y_pred = output_sig.cpu()
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    
    if y_pred != labels.cpu():
        
        if labels.cpu() == 1:
            incorrect_abnormal_list.append(idx)
        else:
            incorrect_normal_list.append(idx)
    label_out = labels.item()
    y_pred_out = y_pred.item()
    print("True label : %d, Predicted label : %d, idx : %s" % (label_out, y_pred_out, idx))
    
    # ============================= #
    # ==== Grad-CAM main lines ==== #
    # ============================= #
    
    score = logit.squeeze() # 예측값 y^c
    score.backward(retain_graph = True) # 예측값 y^c에 대해서 backprop 진행
    
    activations = torch.Tensor(feature_blobs[0]).to(device) # (1, 512, 7, 7), forward activations
    print(torch.Tensor(feature_blobs).shape)
    # print(activations)
    
    
    gradients = backward_feature[0] # (1, 512, 7, 7), backward gradients
    # print(backward_feature)
    # print(gradients)
    b, k, u, v = gradients.size()
    
    # view() 함수에서 -1은 다른 dimension에서 자동적으로 추론되는 것을 의미한다. 
    alpha = gradients.view(b, k, -1).mean(2) # (1, 512, 7*7) => (1, 512), feature map k의 'importance'
    weights = alpha.view(b, k, 1, 1) # (1, 512, 1, 1)
    
    #위에서 지정해준 layer에서의 output인 activations과 backward gradients를 평균한 값인 weights를 곱해준다.
    grad_cam_map = (weights*activations).sum(1, keepdim = True) # alpha * A^k = (1, 512, 7, 7) => (1, 1, 7, 7)
    
    # Apply R e L U
    grad_cam_map = F.relu(grad_cam_map) 
    
    grad_cam_map = F.interpolate(grad_cam_map, size=(224, 224), mode='bilinear', align_corners=False) # (1, 1, 224, 224)
    map_min, map_max = grad_cam_map.min(), grad_cam_map.max()
    grad_cam_map = (grad_cam_map - map_min).div(map_max - map_min).data # (1, 1, 224, 224), min-max scaling

    # grad_cam_map.squeeze() : (224, 224)
    grad_heatmap = cv2.applyColorMap(np.uint8(255 * grad_cam_map.squeeze().cpu()), cv2.COLORMAP_JET) # (224, 224, 3), numpy 
    grad_heatmap = torch.from_numpy(grad_heatmap).permute(2, 0, 1).float().div(255) # (3, 224, 224)
    # print(grad_heatmap)
    b, g, r = grad_heatmap.split(1)
    grad_heatmap = torch.cat([r, g, b]) # (3, 244, 244), opencv's default format is BGR, so we need to change it as RGB format.

    # save_image(grad_heatmap, os.path.join(saved_loc, "%d_%d_%s" % (label_out, y_pred_out, idx_out)))
    
    # print(grad_heatmap.type)
    # print(inputsinputs.cpu().type)
    grad_result = grad_heatmap + inputs.cpu() # (1, 3, 224, 224)
    # print(grad_result.shape)
    grad_result = grad_result.div(grad_result.max()).squeeze() # (3, 224, 224)
    
    # save_image(grad_result, os.path.join(saved_loc, "GradCAM&image_%d.jpg" % (xhch_idx+1)))
    
    
    image_list = []
    
    image_list.append(torch.stack([inputs.squeeze().cpu(), grad_heatmap, grad_result], 0)) # (3, 3, 224, 224)
    
    images = make_grid(torch.cat(image_list, 0), nrow = 3)
    
    # image 저장
    # save_image(images, os.path.join(saved_loc, "%d_%d_%s" % (label_out, y_pred_out, idx)))
    
    # if  batch_idx + 1 == num_result:
    #     break
        
    feature_blobs.clear()
    backward_feature.clear()

feature_blobs.clear()
backward_feature.clear()


torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx : 10626726__2020-12-17__05d.jpg
torch.Size([11, 1, 2048, 7, 7])
torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx : 10626726__2022-01-04__205d.jpg
torch.Size([11, 1, 2048, 7, 7])
torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx : 10626726__2022-01-04__05d.jpg
torch.Size([11, 1, 2048, 7, 7])
torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx : 10836307__2019-12-16__05d.jpg
torch.Size([11, 1, 2048, 7, 7])
torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx : 10836307__2020-12-02__05d.jpg
torch.Size([11, 1, 2048, 7, 7])
torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx : 11265270__2019-02-16__05d.jpg
torch.Size([11, 1, 2048, 7, 7])
torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx : 11265270__2020-06-24__05d.jpg
torch.Size([11, 1, 2048, 7, 7])
torch.Size([1, 2048, 7, 7])
True label : 0, Predicted label : 0, idx

KeyboardInterrupt: 